In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from mlxtend.preprocessing import TransactionEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files 
src = list(files.upload().values())[0] 
open('exploracion.py','wb').write(src) 
import exploracion

Saving exploración.py to exploración.py
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[('ratings',), ('movies',), ('usuarios_sel',), ('peliculas_sel',), ('ratings_final',), ('peliculas_final',), ('full_ratings',), ('df',)]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int

In [4]:
#extraer db de el archivo py
movies=exploracion.dbmovies()
ratings=exploracion.dbratings()

# **Preprocesamiento-Creación de tabla SQL**

In [5]:
# Conectar a la base de datos
conn = sql.connect('/content/drive/MyDrive/Analitica III/MARKETING/db_movies (1)')
##conn = sqlite3.connect('db_movies (1)')
# Borrar la tabla si existe
conn.execute('DROP TABLE IF EXISTS usuarios_sel;')

#######crear tabla con usuarios con más de 50 peliculas vistas y menos de 1000
query = '''
    CREATE TABLE usuarios_sel AS
    SELECT userId,count(*) as cnt_rat
    from ratings
    group by userId
    having cnt_rat <=1000
    order by cnt_rat asc
'''
conn.execute(query)

In [6]:
# Borrar la tabla si existe
conn.execute('DROP TABLE IF EXISTS peliculas_sel;')

####crear tabla con peliculas que han sido vistas por más de 50 usuarios
query2 = '''
    CREATE TABLE peliculas_sel AS
    SELECT movieId,count(*) as cnt_rat
    from ratings
    group by "movieId"
    having cnt_rat>=10
    order by cnt_rat desc
'''
conn.execute(query2)

In [7]:
# Borrar la tabla si existe
conn.execute('DROP TABLE IF EXISTS ratings_final;')

####crear tablas filtradas de peliculas, usuarios y calificaciones

query3 = '''
    CREATE TABLE ratings_final AS
    SELECT A.userId, A.movieId,A.rating,A.timestamp
    from ratings A
    INNER JOIN peliculas_sel B ON B.movieId=A.movieId
    INNER JOIN usuarios_sel C ON C.userId=A.userId;
'''
conn.execute(query3)


In [8]:
# Borrar la tabla si existe
conn.execute('DROP TABLE IF EXISTS peliculas_final;')

####crear tablas filtradas de peliculas, usuarios y calificaciones

query4 = '''
    CREATE TABLE peliculas_final AS
    SELECT A.movieId,A.title,A.genres
    from movies A
    INNER JOIN peliculas_sel B ON B.movieId=A.movieId
'''
conn.execute(query4)

In [9]:
#######crear tabla completa

# Borrar la tabla si existe
conn.execute('DROP TABLE IF EXISTS full_ratings;')

####crear tabla con todo

query5 = '''
    CREATE TABLE full_ratings AS
    SELECT A.userId, B.movieId,A.rating,A.timestamp,B.title,B.genres
    from ratings_final A
    INNER JOIN peliculas_final B ON B.movieId=A.movieId
'''
conn.execute(query5)




In [10]:
#traer el cur del archivo py
cur=exploracion.cur

In [11]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(cur.fetchall())

[('ratings',), ('movies',), ('df',), ('usuarios_sel',), ('peliculas_sel',), ('ratings_final',), ('peliculas_final',), ('full_ratings',)]


verficar tamaño de tablas con filtros


In [12]:
####peliculas

pd.read_sql('select count(*) from movies', conn)


,count(*)
0,9742


In [13]:
pd.read_sql('select count(*) from peliculas_final', conn)

,count(*)
0,2269


In [14]:
##ratings
pd.read_sql('select count(*) from ratings', conn)


,count(*)
0,100836


In [15]:
pd.read_sql('select count(*) from ratings_final', conn)

,count(*)
0,69461


In [16]:
## 3 tablas cruzadas ###
pd.read_sql('select count(*) from full_ratings', conn)

,count(*)
0,69461


In [17]:

ratings=pd.read_sql('select * from full_ratings',conn)
ratings.duplicated().sum() ## al cruzar tablas a veces se duplican registros
ratings.info()
ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69461 entries, 0 to 69460
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     69461 non-null  int64  
 1   movieId    69461 non-null  int64  
 2   rating     69461 non-null  float64
 3   timestamp  69461 non-null  int64  
 4   title      69461 non-null  object 
 5   genres     69461 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 3.2+ MB


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
5,1,70,3.0,964982400,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller
6,1,101,5.0,964980868,Bottle Rocket (1996),Adventure|Comedy|Crime|Romance
7,1,110,4.0,964982176,Braveheart (1995),Action|Drama|War
8,1,151,5.0,964984041,Rob Roy (1995),Action|Drama|Romance|War
9,1,157,5.0,964984100,Canadian Bacon (1995),Comedy|War


In [18]:
df = pd.read_sql('select * from full_ratings', conn)
df


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
69456,609,592,3.0,847220802,Batman (1989),Action|Crime|Thriller
69457,609,742,3.0,847221054,Thinner (1996),Horror|Thriller
69458,609,786,3.0,847221025,Eraser (1996),Action|Drama|Thriller
69459,609,892,3.0,847221080,Twelfth Night (1996),Comedy|Drama|Romance


In [19]:
# Extraer el año utilizando una expresión regular y la función str.extract()
df['year'] = df['title'].str.extract('\((\d{4})\)')

# Eliminar el espacio que separa el título y el año
df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)
df

,userId,movieId,rating,timestamp,title,genres,year
0,1,1,4.0,964982703,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,3,4.0,964981247,Grumpier Old Men,Comedy|Romance,1995
2,1,6,4.0,964982224,Heat,Action|Crime|Thriller,1995
3,1,47,5.0,964983815,Seven (a.k.a. Se7en),Mystery|Thriller,1995
4,1,50,5.0,964982931,"Usual Suspects, The",Crime|Mystery|Thriller,1995
...,...,...,...,...,...,...,...
69456,609,592,3.0,847220802,Batman,Action|Crime|Thriller,1989
69457,609,742,3.0,847221054,Thinner,Horror|Thriller,1996
69458,609,786,3.0,847221025,Eraser,Action|Drama|Thriller,1996
69459,609,892,3.0,847221080,Twelfth Night,Comedy|Drama|Romance,1996


In [20]:
#ratings=pd.read_sql("""select * from full_ratings""",conn)
genres=ratings['genres'].str.split('|')
te = TransactionEncoder()
genres = te.fit_transform(genres)
genres = pd.DataFrame(genres, columns = te.columns_)
genres

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
2,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False
4,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69456,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False
69457,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False
69458,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False
69459,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False


In [21]:
df.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'title', 'genres', 'year'], dtype='object')

In [22]:
### tabla de df se utilizara para modelos #####

df = pd.concat([df,genres], axis=1)
df = df.drop(['genres'], axis=1)
df

,userId,movieId,rating,timestamp,title,year,Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,964982703,Toy Story,1995,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,1,3,4.0,964981247,Grumpier Old Men,1995,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,1,6,4.0,964982224,Heat,1995,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,1,47,5.0,964983815,Seven (a.k.a. Se7en),1995,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
4,1,50,5.0,964982931,"Usual Suspects, The",1995,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69456,609,592,3.0,847220802,Batman,1989,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
69457,609,742,3.0,847221054,Thinner,1996,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
69458,609,786,3.0,847221025,Eraser,1996,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
69459,609,892,3.0,847221080,Twelfth Night,1996,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [23]:
df.to_sql('df', conn, if_exists='replace')

69461

In [24]:
df_1 = pd.read_sql('select * from full_ratings', conn)
df_1

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
69456,609,592,3.0,847220802,Batman (1989),Action|Crime|Thriller
69457,609,742,3.0,847221054,Thinner (1996),Horror|Thriller
69458,609,786,3.0,847221025,Eraser (1996),Action|Drama|Thriller
69459,609,892,3.0,847221080,Twelfth Night (1996),Comedy|Drama|Romance


In [25]:
#Importar csv a drive
df_1.to_csv('/content/drive/MyDrive/Analitica III/MARKETING/df.csv', index=False, mode='w')